In [24]:
import torch
import torchvision

In [30]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor()
     , torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [31]:
trainset = torchvision.datasets.MNIST(root='./data'
                                            , train=True
                                            , download=True
                                            , transform=transform )

testset = torchvision.datasets.MNIST(root='./data'
                                            , train=False
                                            , download=True
                                            , transform=transform )

In [32]:
trainloader = torch.utils.data.DataLoader(trainset, shuffle=True, batch_size=4, num_workers=2)

In [57]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.cn1  = torch.nn.Conv2d(1, 6, 5)
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.cn2  = torch.nn.Conv2d(6, 16, 5)
        #self.dl1  = torch.nn.Linear(16 * 5 * 5, 120)
        self.dl1  = torch.nn.Linear(256, 120)
        self.dl2  = torch.nn.Linear(120, 84)
        self.dl3  = torch.nn.Linear(84, 10)
    def forward(self, x):
        x = self.cn1(x)
        x = self.pool( torch.nn.functional.relu(x) )
        x = self.cn2(x)
        x = self.pool( torch.nn.functional.relu(x) )
        x = x.view(-1, self.num_flat_features(x))
        x = torch.nn.functional.relu( self.dl1(x) )
        x = torch.nn.functional.relu( self.dl2(x) )
        x = self.dl3( x )
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
net = Net()

In [58]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [60]:
for epoch in range(1):
    for batch_number, data in enumerate(trainloader):
        optimizer.zero_grad()
        x, y = data
        #x = x.view(4, 1, 28, 28)
        x, y = torch.autograd.Variable(x), torch.autograd.Variable(y_)
        y_ = net(x)
        loss = criterion(y_, y)
        loss.backward()
        optimizer.step()
        if batch_number % 2000 == 1999:
            print(batch_number)

1999
3999
5999
7999
9999
11999
13999


In [61]:
testloader = torch.utils.data.DataLoader(testset, shuffle=True, batch_size=4, num_workers=2)

In [67]:
iterator = iter(testloader)
x, y = iterator.next()
y_ = net( torch.autograd.Variable(x) )
y_ = torch.max(y_.data, 1)[1]
print('prediction: ', y_)
print('ground truth:', y)


prediction:  
 0
 3
 5
 0
[torch.LongTensor of size 4]

ground truth: 
 0
 3
 5
 0
[torch.LongTensor of size 4]

